In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# CONSTANTS
IMAGE_WIDTH = 96
IMAGE_HEIGHT = 96
IMAGE_NUMBER_CHANNELS = 3
NUM_CLASSES = 120

Using TensorFlow backend.


In [2]:
x_train = np.load('dogs_x_train.npy')
y_train = np.load('dogs_y_train.npy')
#x_valid = np.load('x_valid.npy')
#y_valid = np.load('y_valid.npy')
#x_test = np.load('x_test.npy')
#y_test = np.load('y_test.npy')

#Normalize data
x_train = x_train.astype('float32')
#x_valid = x_valid.astype('float32')
#x_test = x_test.astype('float32')
x_train = x_train / 255
#x_valid = x_valid / 255
#x_test = x_test / 255

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, NUM_CLASSES)
#y_valid = to_categorical(y_valid, NUM_CLASSES)
#y_test = to_categorical(y_test, NUM_CLASSES)

#MNIST resolution
img_rows, img_cols, channels = IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_NUMBER_CHANNELS

from tensorflow.keras import backend as K

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], channels, img_rows, img_cols)
    #x_valid = x_valid.reshape(x_valid.shape[0], channels, img_rows, img_cols)
    #x_test = x_test.reshape(x_test.shape[0], channels, img_rows, img_cols)
    input_shape = (channels, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channels)
    #x_valid = x_valid.reshape(x_valid.shape[0], img_rows, img_cols, channels)
    #x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channels)
    input_shape = (img_rows, img_cols, channels)

In [3]:
'''# Data load and splitting
X = np.load("X.npy")
Y = np.load("Y.npy")

Y = np_utils.to_categorical(Y, NUM_CLASSES)

mask = np.random.rand(len(X)) < 0.9

X_trn, X_tst = X[mask], X[~mask]
Y_trn, Y_tst = Y[mask], Y[~mask]'''

# Adaptacion
X = x_train
Y = y_train

mask = np.random.rand(len(X)) < 0.9

X_trn, X_tst = X[mask], X[~mask]
Y_trn, Y_tst = Y[mask], Y[~mask]

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_trn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

# Let's compile the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://10.240.1.2:8470'))
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.240.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 1518520649310485134)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14552200333382919537)
INFO:tensorflow:*** Available Device: _DeviceAttri

In [ ]:
'''callbacks = [EarlyStopping(monitor='val_acc', patience=6)]

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_trn)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    datagen.flow(X_trn, Y_trn, batch_size=64),
    epochs=150,
    validation_data=(X_tst, Y_tst),
    workers=4,
    callbacks=callbacks)'''

'callbacks = [EarlyStopping(monitor=\'val_acc\', patience=6)]\n\n# This will do preprocessing and realtime data augmentation:\ndatagen = ImageDataGenerator(\n    featurewise_center=False,  # set input mean to 0 over the dataset\n    samplewise_center=False,  # set each sample mean to 0\n    featurewise_std_normalization=False,  # divide inputs by std of the dataset\n    samplewise_std_normalization=False,  # divide each input by its std\n    zca_whitening=False,  # apply ZCA whitening\n    zca_epsilon=1e-06,  # epsilon for ZCA whitening\n    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)\n    # randomly shift images horizontally (fraction of total width)\n    width_shift_range=0.1,\n    # randomly shift images vertically (fraction of total height)\n    height_shift_range=0.1,\n    # set mode for filling points outside the input boundaries\n    fill_mode=\'nearest\',\n    cval=0.,  # value used for fill_mode = "constant"\n    horizontal_flip=True,  # random

In [ ]:
model.fit(X_trn, Y_trn,
              batch_size=64,
              epochs=150,
              validation_data=(X_tst, Y_tst),
              shuffle=True, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=6)])

Train on 18549 samples, validate on 2031 samples
Epoch 1/150
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(8,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(8, 96, 96, 3), dtype=tf.float32, name='conv2d_input_10'), TensorSpec(shape=(8, 120), dtype=tf.float32, name='activation_5_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'beta_1': 0.8999999761581421, 'amsgrad': False, 'decay': 0.0, 'beta_2': 0.9990000128746033, 'lr': 0.0010000000474974513, 'epsilon': 1e-07}
INFO:tensorflow:Remapping placeholder for conv2d_input
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.Adam object at 0x7fb5c80af7f0> []
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 5.2643516063690186 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU

18549/18549 [==============================] - 13s 709us/sample - loss: 4.7808 - acc: 0.0119 - val_loss: 4.7814 - val_acc: 0.0138
Epoch 6/150
18549/18549 [==============================] - 13s 691us/sample - loss: 4.7806 - acc: 0.0111 - val_loss: 4.7813 - val_acc: 0.0138
Epoch 7/150
17344/18549 [===========================>..] - ETA: 0s - loss: 4.7802 - acc: 0.0121